##### Fluid Dynamics-Inspired Emotional Analysis & Sentiment Flow for Argumentation Mining (AM)
- claculate flow in each datasets to get insights


In [2]:
import shutil
import pandas as pd
from SentimentFlow import SpeechProcessor
from SentimentFlow import SentimentFlowCalculator
#https://github.com.mcas.ms/dpicca/SentimentFlow
pd.set_option("display.max_colwidth", None)
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
import os
import spacy
from nrclex import NRCLex

import pandas as pd
import spacy
from nrclex import NRCLex
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from SentimentFlow import SpeechProcessor
from SentimentFlow import SentimentFlowCalculator

from textblob import TextBlob

- https://aclanthology.org/2024.mathnlp-1.2.pdf

# Argumentative Microtext Part 1

we've done a extensive pre-processing on microtext to extract all layers of annotations. The focus is stance classifcation 

In [3]:
df = pd.read_json("../data/dfMT-PC_SA_CP_topicID_AduType.json") 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112 entries, 0 to 111
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   EDU        112 non-null    object
 1   adu_type   112 non-null    object
 2   topic_id   89 non-null     object
 3   Relations  112 non-null    object
 4   Label      112 non-null    object
 5   STANCE     89 non-null     object
dtypes: object(6)
memory usage: 6.1+ KB


In [5]:
df[0:10]   

,EDU,adu_type,topic_id,Relations,Label,STANCE
0,"[Yes, it's annoying and cumbersome to separate your rubbish properly all the time., Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins., But still Germany produces way too much rubbish, and too many resources are lost when what actually should be separated and recycled is burnt., We Berliners should take the chance and become pioneers in waste separation!]","[opp, opp, pro, pro, pro]",waste_separation,"[[a1, a5, reb], [a2, a1, sup], [a3, c1, und], [a4, c3, add]]","[0, 0, 0, 0, 1]",pro
1,"[One can hardly move in Friedrichshain or Neukölln these days without permanently scanning the ground for dog dirt., And when bad luck does strike and you step into one of the many 'land mines' you have to painstakingly scrape the remains off your soles., Higher fines are therefore the right measure against negligent, lazy or simply thoughtless dog owners., Of course, first they'd actually need to be caught in the act by public order officers,, but once they have to dig into their pockets, their laziness will sure vanish!]","[pro, pro, pro, opp, pro]",higher_dog_poo_fines,"[[a1, a3, sup], [a2, a3, sup], [a4, a3, reb], [a5, c4, und]]","[0, 0, 1, 0, 0]",pro
2,"[Health insurance companies should not cover treatment in complementary medicine, unless the promised effect and its medical benefit have been concretely proven., Yet this very proof is lacking in most cases., Patients do often report relief of their complaints after such treatments., But as long as it is unclear as to how this works, the funds should rather be spent on therapies where one knows with certainty.]","[pro, opp, pro, opp, pro]",health_insurance_cover_complementary_medicine,"[[a2, a1, reb], [a3, a2, reb], [a4, a1, reb], [a5, c4, und]]","[1, 0, 0, 0, 0]",con
3,"[Of course there are a number of programmes in public broadcasting that are not worth the licencing fee,, and others, such as “Musikantenstadl” and soap operas, are only interesting to certain audiences., Nevertheless, everybody should contribute to the funding of the public broadcasters in equal measure,, for we need general and independent media., After all we want to get our view of the world neither through the lens of the government nor through that of rich media entrepreneurs.]","[opp, opp, pro, pro, pro]",public_broadcasting_fees_on_demand,"[[a1, a3, reb], [a2, a3, reb], [a4, a3, sup], [a5, a4, sup]]","[0, 0, 1, 0, 0]",con
4,"[Intelligence services must urgently be regulated more tightly by parliament;, this should be clear to everyone after the disclosures of Edward Snowden., Granted, those concern primarily the British and American intelligence services,, but the German services evidently do collaborate with them closely., Their tools, data and expertise have been used to keep us under surveillance for a long time.]","[pro, pro, opp, pro, pro]",stricter_regulation_of_intelligence_services,"[[a2, a1, sup], [a3, c2, und], [a4, c3, und], [a5, a4, sup]]","[1, 0, 0, 0, 0]",pro
5,"[The death penalty is a legal means that as such is not practicable in Germany., For one thing, inviolable human dignity is anchored in our constitution,, and furthermore no one may have the right to adjudicate upon the death of another human being., Even if many people think that a murderer has already decided on the life or death of another person,, this is precisely the crime that we should not repay with the same.]","[pro, pro, pro, opp, pro]",introduce_capital_punishment,"[[a2, a1, sup], [a3, a1, sup], [a4, a1, reb], [a5, c4, und]]","[1, 0, 0, 0, 0]",con
6,"[Dog dirt on the pavement requires my complete, ground-focused attention on my way to work., This results in a lack of alertness in the road traffic comparable to a minor offense of the traffic regulations., That's why causing such obstacles must to be more urgently punished., Dog owners may think that dog dirt is only a harmless secretion of an animal like any other., Yet then they misj

### A-NRC for emotion analysis  (Davida Package)

In [6]:
def calculate_nrc(text):
    """Calculate NRC emotion frequencies for a given text."""
    nrc_sentiment = NRCLex(text)
    affect_dict = nrc_sentiment.affect_frequencies

    base_emotions = ['fear', 'anger', 'trust', 'surprise', 
                     'positive', 'negative', 'sadness', 'disgust', 'joy', "anticipation"]
    
    for emotion in base_emotions:
        if emotion not in affect_dict:
            affect_dict[emotion] = 0.0

    return {emotion: affect_dict[emotion] for emotion in base_emotions}

In [7]:
print(df.EDU[0],"\n", df.adu_type[0])

["Yes, it's annoying and cumbersome to separate your rubbish properly all the time.", 'Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins.', 'But still Germany produces way too much rubbish', 'and too many resources are lost when what actually should be separated and recycled is burnt.', 'We Berliners should take the chance and become pioneers in waste separation!'] 
 ['opp', 'opp', 'pro', 'pro', 'pro']


In [8]:
calculate_nrc(df.EDU[0][4])#which feaure are usefull for stance classfication= 

{'fear': 0.0,
 'anger': 0.0,
 'trust': 0.0,
 'surprise': 0.3333333333333333,
 'positive': 0.0,
 'negative': 0.3333333333333333,
 'sadness': 0.0,
 'disgust': 0.3333333333333333,
 'joy': 0.0,
 'anticipation': 0.0}

# Argumentative Microtext Part 2

In [9]:
df2 = pd.read_json("../data/dfMT2-PC_SA_CP_topicID_AduType.json") 

In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 170
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   EDU        171 non-null    object
 1   adu_type   171 non-null    object
 2   topic_id   171 non-null    object
 3   Relations  171 non-null    object
 4   Label      171 non-null    object
 5   STANCE     171 non-null    object
dtypes: object(6)
memory usage: 9.4+ KB


In [11]:
df3 = pd.read_json("../data/df_IBM.json")   #work wore on extraction 

In [12]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 0 to 54
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   topic_id         55 non-null     object
 1   EDU              55 non-null     object
 2   adu_type         55 non-null     object
 3   targetsRelation  55 non-null     object
 4   Compatible       55 non-null     object
dtypes: object(5)
memory usage: 2.6+ KB


In [13]:
df3.head(1)

topic_id  \
0  the sale of violent video games to minors   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [14]:
def convert_list(lst):
    return ['pro' if item == 'PRO' else 'opp' for item in lst]

df3['adu_type'] = df3['adu_type'].apply(convert_list)

In [15]:
def handle_mismatched_rows(df):
    mismatched_rows = df[df.apply(lambda row: len(row['EDU']) != len(row['adu_type']), axis=1)]
    
    #if not mismatched_rows.empty:
    #    print(f"Found {len(mismatched_rows)} rows with mismatched list lengths.")
    #    print(mismatched_rows)
    
    df = df[df.apply(lambda row: len(row['EDU']) == len(row['adu_type']), axis=1)]
    
    return df

In [16]:
df2_new = handle_mismatched_rows(df2)

In [17]:
df2_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, 0 to 170
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   EDU        144 non-null    object
 1   adu_type   144 non-null    object
 2   topic_id   144 non-null    object
 3   Relations  144 non-null    object
 4   Label      144 non-null    object
 5   STANCE     144 non-null    object
dtypes: object(6)
memory usage: 7.9+ KB


# ### B-Senticnet for emotion analysis (Davida Package)

In [18]:
def calculate_senticnet(text):
    processor = SpeechProcessor('../../data/senticnet/senticnet2.tsv')
    df_sc_sentiment = processor.process_texts(pd.Series(text)) 
    print(df_sc_sentiment)
    df_sc_sentiment.columns = df_sc_sentiment.columns.str.replace('#', '_')
    
    expected_keys = ["INTROSPECTION_ecstasy", "TEMPER", "SENSITIVITY", "ATTITUDE", "TEMPER_calmness", "POLARITY"]
    dict_sc_sentiment = {key: df_sc_sentiment[key].values[0] if key in df_sc_sentiment else 0 for key in expected_keys}

    return dict_sc_sentiment


In [19]:
calculate_senticnet(df.EDU[0][0])

03-Apr-25 16:56:36 - Starting to process


Processing texts:   0%|          | 0/1 [00:00<?, ?it/s]

03-Apr-25 16:56:37 - Saving results to results/processed_texts.csv


                                                                                text  \
0  Yes, it's annoying and cumbersome to separate your rubbish properly all the time.   

   INTROSPECTION#ecstasy  TEMPER  SENSITIVITY  ATTITUDE  TEMPER#calmness  \
0                  0.856   -0.33       -0.275    -0.596            0.552   

   POLARITY  
0    -0.078  


{'INTROSPECTION_ecstasy': 0.856,
 'TEMPER': -0.33,
 'SENSITIVITY': -0.275,
 'ATTITUDE': -0.596,
 'TEMPER_calmness': 0.552,
 'POLARITY': -0.078}

In [30]:
df.EDU[0]

["Yes, it's annoying and cumbersome to separate your rubbish properly all the time.",
 'Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins.',
 'But still Germany produces way too much rubbish',
 'and too many resources are lost when what actually should be separated and recycled is burnt.',
 'We Berliners should take the chance and become pioneers in waste separation!']

In [32]:
processor = SpeechProcessor('../../data/senticnet/senticnet2.tsv')
df_sc_sentiment = processor.process_texts(pd.Series(df.EDU[0])) 
df_sc_sentiment.columns = df_sc_sentiment.columns.str.replace('#', '_')   
print(df_sc_sentiment)
flow_calculator = SentimentFlowCalculator()
results = flow_calculator.calculate_navier_stocker_for_texts(df_sc_sentiment)
results

03-Apr-25 17:05:35 - Starting to process


Processing texts:   0%|          | 0/5 [00:00<?, ?it/s]

03-Apr-25 17:05:37 - Saving results to results/processed_texts.csv
03-Apr-25 17:05:37 - Starting to calculate Navier-Stocker...


                                                                                                    text  \
0                      Yes, it's annoying and cumbersome to separate your rubbish properly all the time.   
1  Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins.   
2                                                        But still Germany produces way too much rubbish   
3          and too many resources are lost when what actually should be separated and recycled is burnt.   
4                           We Berliners should take the chance and become pioneers in waste separation!   

   INTROSPECTION_ecstasy  TEMPER  SENSITIVITY  ATTITUDE  TEMPER_calmness  \
0                 0.8560   -0.33       -0.275    -0.596            0.552   
1                 0.9610    0.00        0.000    -0.393            0.000   
2                 0.8250    0.00        0.000    -0.596            0.000   
3                 0.0000    0.00        0.000  

Calculating Navier-Stocker for texts:   0%|          | 0/5 [00:00<?, ?it/s]

h:\moha\NLP\AM\SA\.venv\lib\site-packages\SentimentFlow\sentiment_analysis.py:250: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  s = odeint(self._differential_equation, s0, t, args=(speech_info,))
03-Apr-25 17:05:37 - Finished calculating texts.
03-Apr-25 17:05:37 - Processed 5 texts.
03-Apr-25 17:05:37 - Saving results to results/navier_stocker_text_results.json


{0: [{'text': ("Yes, it's annoying and cumbersome to separate your rubbish properly all the time.",
    "Yes, it's annoying and cumbersome to separate your rubbish properly all the time."),
   'simulation': array([[-5.96000000e-01,  8.56000000e-01,  0.00000000e+00,
           -7.80000000e-02, -2.75000000e-01,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00, -3.30000000e-01,
            5.52000000e-01],
          [-1.63575859e+06,  1.14346568e+06, -2.85831766e-01,
            6.48547461e+00, -3.09164562e-01, -9.98397970e-02,
           -1.04553669e-01,  3.88781699e-02, -3.77871112e-01,
            1.18225487e+00]]),
   'emotion dimension': Index(['ATTITUDE', 'INTROSPECTION_ecstasy', 'INTROSPECTION_loathing',
          'POLARITY', 'SENSITIVITY', 'SENSITIVITY_eagerness',
          'SENSITIVITY_enthusiasm', 'SENSITIVITY_loathing', 'TEMPER',
          'TEMPER_calmness'],
         dtype='object')}],
 1: [{'text': ('Three different bin bags stink away in the kitchen and have to b

In [28]:
pd.Series(df.EDU[0])

0                        Yes, it's annoying and cumbersome to separate your rubbish properly all the time.
1    Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins.
2                                                          But still Germany produces way too much rubbish
3            and too many resources are lost when what actually should be separated and recycled is burnt.
4                             We Berliners should take the chance and become pioneers in waste separation!
dtype: object

In [25]:
df.EDU[0]

["Yes, it's annoying and cumbersome to separate your rubbish properly all the time.",
 'Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins.',
 'But still Germany produces way too much rubbish',
 'and too many resources are lost when what actually should be separated and recycled is burnt.',
 'We Berliners should take the chance and become pioneers in waste separation!']

In [24]:
calculate_Flow(df.EDU[0])

03-Apr-25 16:57:40 - Starting to process


Processing texts:   0%|          | 0/5 [00:00<?, ?it/s]

03-Apr-25 16:57:41 - Saving results to results/processed_texts.csv
03-Apr-25 16:57:41 - Starting to calculate Navier-Stocker...


Calculating Navier-Stocker for texts:   0%|          | 0/5 [00:00<?, ?it/s]

h:\moha\NLP\AM\SA\.venv\lib\site-packages\SentimentFlow\sentiment_analysis.py:250: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  s = odeint(self._differential_equation, s0, t, args=(speech_info,))
03-Apr-25 16:57:41 - Finished calculating texts.
03-Apr-25 16:57:41 - Processed 5 texts.
03-Apr-25 16:57:41 - Saving results to results/navier_stocker_text_results.json


{0: [{'text': ("Yes, it's annoying and cumbersome to separate your rubbish properly all the time.",
    "Yes, it's annoying and cumbersome to separate your rubbish properly all the time."),
   'simulation': array([[-5.96000000e-01,  8.56000000e-01,  0.00000000e+00,
           -7.80000000e-02, -2.75000000e-01,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00, -3.30000000e-01,
            5.52000000e-01],
          [-1.63575859e+06,  1.14346568e+06, -2.85831766e-01,
            6.48547461e+00, -3.09164562e-01, -9.98397970e-02,
           -1.04553669e-01,  3.88781699e-02, -3.77871112e-01,
            1.18225487e+00]]),
   'emotion dimension': Index(['ATTITUDE', 'INTROSPECTION_ecstasy', 'INTROSPECTION_loathing',
          'POLARITY', 'SENSITIVITY', 'SENSITIVITY_eagerness',
          'SENSITIVITY_enthusiasm', 'SENSITIVITY_loathing', 'TEMPER',
          'TEMPER_calmness'],
         dtype='object')}],
 1: [{'text': ('Three different bin bags stink away in the kitchen and have to b